# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.  

__Информация о данных__  
*id* — уникальный идентификатор скважины;  
*f0, f1, f2* — три признака точек (неважно, что они означают, но сами признаки значимы);  
*product* — объём запасов в скважине (тыс. баррелей).  

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from IPython.display import display
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
df0 = pd.read_csv('/datasets/geo_data_0.csv')
df1 = pd.read_csv('/datasets/geo_data_1.csv')
df2 = pd.read_csv('/datasets/geo_data_2.csv')
dfs = [df0,df1,df2]
for df in dfs:
    df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8

Проверим данные на пропуски и дубликаты. Удалим ненужный столбец 'id'.

In [3]:
for df in dfs:
    print(df.isna().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


In [4]:
for df in dfs:
    print(df.duplicated().sum())
for df in dfs:
    print(len(df['id'].unique()))

0
0
0
99990
99996
99996


In [5]:
for i in range(3):
    dfs[i] = dfs[i].drop_duplicates(subset=['id']).reset_index(drop=True)
    dfs[i] = dfs[i].drop(['id'], axis=1)

## Обучение и проверка модели

Разобьем выборку на 25/75 на валидационную и обучающую соответственно. Обучим модель и расчитаем значения запасов.

In [6]:
def model_pr(data):
    features = data.drop(['product'], axis = 1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)
    model = LinearRegression()
    model.fit(features_train, target_train)
    return pd.Series(model.predict(features_valid)), target_valid
    

In [7]:
pred0, target0 = model_pr(dfs[0])
pred1, target1 = model_pr(dfs[1])
pred2, target2 = model_pr(dfs[2])

Напишем функцию для расчета среднеквадратической ошибки плюс она будет показывать средний прогнозируемый объем сырья.

In [8]:
def calc_rmse (answers, predictions):
    mse = mean_squared_error(answers, predictions)
    print('Средний прогнозируемый запас сырья, тыс. баррелей:', predictions.mean())
    print('RMSE:', mse ** 0.5)

In [9]:
print('Регион 1')
calc_rmse(target0, pred0)

Регион 1
Средний прогнозируемый запас сырья, тыс. баррелей: 92.78915638280621
RMSE: 37.853527328872964


In [10]:
print('Регион 2')
calc_rmse(target1, pred1)

Регион 2
Средний прогнозируемый запас сырья, тыс. баррелей: 69.17831957030431
RMSE: 0.892059264771702


In [11]:
print('Регион 3')
calc_rmse(target2, pred2)

Регион 3
Средний прогнозируемый запас сырья, тыс. баррелей: 94.86572480562035
RMSE: 40.07585073246016


Наименьшая RMSE у региона 2, набольшеий запас у региона 3. Далее расчитаем прибыль.

## Подготовка к расчёту прибыли

Расчитаем минимальный необходимый запас скважины, чтобы выйти в 0 при имеющемся бюджете.

In [12]:
reg_budget = 10000000000
barell_price = 450
wells_num = 200
min_capacity = reg_budget / barell_price / 1000 / wells_num
print(min_capacity)

111.11111111111111


Выше среднего из любого региона. Придется отбирать скважины с наибольшим запасом. Напишем функцию, которая принимает на вход данные об объеме сырья в регионе и по 200 скважиам с наибольшим запасом расчитывает выручку в регионе.

In [13]:
def revenue(target, probabilities):
    target = pd.Series(target).reset_index(drop=True)
    probabilities = pd.Series(probabilities).reset_index(drop=True)
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:wells_num]
    mlrd = 1000000000
    return ((barell_price*1000* selected.sum()) - reg_budget) / mlrd

In [14]:
revenue(target0, pred0)

3.3651872377002867

In [15]:
revenue(target1, pred1)

2.4150866966815108

In [16]:
revenue(target2, pred2)

2.5012838532820627

Наибольшая потенциальная выручка у региона 1, наименьшая у региона 2. Оставим лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью. Применим Bootstrap с 1000 выборок.

## Расчёт прибыли и рисков 

In [17]:
def bootstrap (target, predictions):
    values = []
    for i in range(1000):
        target_subsample = pd.Series(target).reset_index(drop=True).sample(n=500, replace=True, random_state=i)
        probs_subsample = pd.Series(predictions)[target_subsample.index]
        rev = revenue(target_subsample, probs_subsample)
        values.append(rev)
    risk = 0.025
    values = pd.Series(values)
    lower = values.quantile(risk)
    confidence_interval = (values.quantile(0.025), values.quantile(0.975))
    mean = values.mean()

    print("Средняя выручка:", mean)
    print("2.5%-квантиль:", lower)
    print("Доверительный интервал:", confidence_interval)
    print("Риск убытка:", (values < 0).mean())

In [18]:
bootstrap(target0, pred0)

Средняя выручка: 0.39210870898443556
2.5%-квантиль: -0.15031343756955062
Доверительный интервал: (-0.15031343756955062, 0.8740908896288031)
Риск убытка: 0.071


In [19]:
bootstrap(target1, pred1)

Средняя выручка: 0.47129712687330044
2.5%-квантиль: 0.07875522386216341
Доверительный интервал: (0.07875522386216341, 0.8703911603084294)
Риск убытка: 0.013


In [20]:
bootstrap(target2, pred2)

Средняя выручка: 0.3275970991940312
2.5%-квантиль: -0.17393194951400787
Доверительный интервал: (-0.17393194951400787, 0.8318109823786615)
Риск убытка: 0.111


Исходя из результатов и анализа расчета прибыли и рисков, наиболее продуктивным является второй регион по средней выручке, с наименьшим риском убытков, чем у остальных регионов = 0,013. Я бы порекомендовал выбрать второй регион.